In [1]:
#!pip install torch torchvision

Imports

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import Dataset

Variables/Hyperparameters

In [3]:
seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [45]:
input_size = 128
batch_size = 512
epochs = 10
learning_rate = 1e-3

source_folder = '/content/drive/My Drive/MachineLearning/AtariPinball/Data'
data_file = 'pinballmemory.csv'
destination_folder = '/content/drive/My Drive/MachineLearning/AtariPinball/Model'

Create Dataset for Atari Pinball RAM

In [18]:
class RAMDataset(Dataset):

    def __init__(self, csv_file):
        self.train = pd.read_csv(csv_file)

        self.training = "label" in self.train.columns
        self.train_x = self.train if not self.training else self.train.drop("label", axis=1)

    def __getitem__(self, idx):        
        X = np.asarray(self.train_x.iloc[idx], dtype=np.float32)
        return torch.from_numpy(X)

    def __len__(self):
        return len(self.train)

In [19]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

train_dataset = RAMDataset(os.path.join(source_folder, data_file))
print(train_dataset[5])

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)

tensor([  0.,   0.,   0.,   8., 207., 176., 112.,   0.,   2.,   0.,  16.,   0.,
         15.,   0.,   0.,   1.,   0.,   0.,   0.,  64.,   0.,   0.,   0.,   0.,
        193.,   1.,  88.,   0.,  15.,  43., 248., 118.,   0.,   1.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0., 255.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0., 149., 149., 132.,  66.,  67.,   8.,
         80.,  80.,  80.,   8.,   8., 115.,  94.,  94.,  48.,   8.,   0.,   8.,
          0.,   0.,   0.,   3.,   0., 254.,  60., 252.,   0., 252.,   0., 252.,
          0., 252.,  32., 253.,  96., 253., 176., 253., 240., 253.,   0.,   0.,
        112., 115., 238., 141., 242.,  80., 141., 242.,   8., 141., 242.,  94.,
        141., 242.,   8., 141., 135., 246., 229., 244.])


In [43]:
class AE(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.encoder_hidden_layer = nn.Linear(
            in_features=kwargs["input_shape"], out_features=64
        )
        self.encoder_hidden_layer2 = nn.Linear(
            in_features=64, out_features=32
        )
        self.encoder_output_layer = nn.Linear(
            in_features=32, out_features=64
        )
        self.decoder_hidden_layer = nn.Linear(
            in_features=64, out_features=128
        )
        self.decoder_output_layer = nn.Linear(
            in_features=128, out_features=kwargs["input_shape"]
        )

    def forward(self, features):
        activation = self.encoder_hidden_layer(features)
        activation = torch.relu(activation)
        activation = self.encoder_hidden_layer2(activation)
        activation = torch.relu(activation)
        code = self.encoder_output_layer(activation)
        code = torch.relu(code)
        activation = self.decoder_hidden_layer(code)
        activation = torch.relu(activation)
        activation = self.decoder_output_layer(activation)
        reconstructed = torch.relu(activation)
        return reconstructed

In [44]:
#  use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# create a model from `AE` autoencoder class
# load it to the specified device, either gpu or cpu
model = AE(input_shape=input_size).to(device)

# create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# mean-squared error loss
criterion = nn.MSELoss()

In [46]:
for epoch in range(epochs):
    loss = 0

    for batch_features in train_loader:
        # reshape mini-batch data to [N, input_size] matrix
        # load it to the active device
        batch_features = batch_features.view(-1, input_size).to(device)
        
        # reset the gradients back to zero
        # PyTorch accumulates gradients on subsequent backward passes
        optimizer.zero_grad()
        
        # compute reconstructions
        outputs = model(batch_features)
        
        # compute training reconstruction loss
        train_loss = criterion(outputs, batch_features)
        
        # compute accumulated gradients
        train_loss.backward()
        
        # perform parameter update based on current gradients
        optimizer.step()
        
        # add the mini-batch training loss to epoch loss
        loss += train_loss.item()
    
    # compute the epoch training loss
    loss = loss / len(train_loader)
    
    # display the epoch training loss
    print("epoch : {}/{}, recon loss = {:.8f}".format(epoch + 1, epochs, loss))

epoch : 1/10, recon loss = 4557.04899924
epoch : 2/10, recon loss = 3236.50642588
epoch : 3/10, recon loss = 2920.00892534
epoch : 4/10, recon loss = 1835.69997226
epoch : 5/10, recon loss = 1205.15831129
epoch : 6/10, recon loss = 1194.89035443
epoch : 7/10, recon loss = 1185.65398964
epoch : 8/10, recon loss = 1181.53642523
epoch : 9/10, recon loss = 1178.71072682
epoch : 10/10, recon loss = 1179.86220798


In [47]:
with torch.no_grad():
    for batch_features in train_loader:
        batch_features = batch_features[0]
        batch_features = batch_features.to(device)
        test_examples = batch_features.view(-1, input_size)
        reconstruction = model(test_examples)
        break

In [51]:
reconstruction[0].cpu().numpy()

array([  0.       ,   0.       ,   0.       ,   7.6881323, 206.25645  ,
       175.49516  , 109.7171   ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       , 131.98079  ,   5.43821  , 191.14342  ,
         0.       , 251.88797  ,   0.691317 ,  15.223038 ,  42.83775  ,
       245.65416  , 116.84229  ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       , 253.91846  ,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
        16.83022  ,   0.       ,   0.       ,   0.       , 158.97246  ,
        29.07904  ,  21.91979  ,  78.96609  , 126.61489  , 221.76933  ,
        20.318096 ,   0.       ,   0.       ,   0.       ,   0.       ,
         0.       , 148.22499  ,  50.340656 ,  83.261795 ,  42.1

In [53]:
data = reconstruction[0].cpu().numpy()
#data = data.astype(np.float32) / 1 # normalize the data to 0 - 1
#data = 255 * data # Now scale by 255
#to enable rounding, add 0.5 to everything, then truncate by casting
data = data + 0.5
scaled_data = data.astype(np.uint8)
print(scaled_data)

[  0   0   0   8 206 175 110   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0 132   5 191   0 252   1  15  43 246 117   0   0   0   0
   0   0   0   0   0   0 254   0   0   0   0   0   0   0  17   0   0   0
 159  29  22  79 127 222  20   0   0   0   0   0 148  50  83  42  67   0
  77  79  77  11   4 114  97   0  48   4 125   4   0   0   0   3   0 253
  60 250   0 251   0 250   0 250  13 251  76 252 144   0 208 251   7   0
 111 107 224 140 240  77 140 241  10 140 240  99 140 240   5 141 135 245
 228 242]


In [56]:
print(train_dataset[0].numpy().astype(np.uint8))

[  0   0   0   8 207 176 112   0   4   0  16   0  13   0   0   1   0   0
   0  64   0   0   0   0 193   1  74   0  15  43 248 118   0   1   0   0
   0   0   0   0   0   0 255   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0 149 149 132  66  67   8
  80  80  80   8   8 115 101 101  48   8   0   8   0   0   0   3   0 254
  60 252   0 252   0 252   0 252   0 253  64 253 176 253 240 253   0   0
 112 115 238 141 242  80 141 242   8 141 242 101 141 242   8 141 135 246
 229 244]


In [ ]:
# test_dataset = torchvision.datasets.MNIST(
#     root="~/torch_datasets", train=False, transform=transform, download=True
# )

# test_loader = torch.utils.data.DataLoader(
#     test_dataset, batch_size=10, shuffle=False
# )

# test_examples = None

# with torch.no_grad():
#     for batch_features in test_loader:
#         batch_features = batch_features[0]
#         batch_features = batch_features.to(device)
#         test_examples = batch_features.view(-1, 784)
#         reconstruction = model(test_examples)
#         break

In [ ]:
# with torch.no_grad():
#     number = 10
#     plt.figure(figsize=(20, 4))
#     for index in range(number):
#         # display original
#         ax = plt.subplot(2, number, index + 1)
#         plt.imshow(test_examples[index].cpu().numpy().reshape(28, 28))
#         plt.gray()
#         ax.get_xaxis().set_visible(False)
#         ax.get_yaxis().set_visible(False)

#         # display reconstruction
#         ax = plt.subplot(2, number, index + 1 + number)
#         plt.imshow(reconstruction[index].cpu().numpy().reshape(28, 28))
#         plt.gray()
#         ax.get_xaxis().set_visible(False)
#         ax.get_yaxis().set_visible(False)
#     plt.show()